In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("MinTemp")
sc = SparkContext(conf=conf)

In [2]:
sc

<SparkContext master=local appName=MinTemp>

In [3]:
lines_rdd = sc.textFile("16_1800.csv")

In [14]:
STATION_ID = 0
DATA_TYPE =2
TEMP = 3

In [19]:
def split_line(line):
    cols = line.split(",")
    station = str(cols[STATION_ID])
    which_data = str(cols[DATA_TYPE])
    temp = float(cols[TEMP])* 0.1 * (9.0 / 5.0) + 32.0
    return (station,which_data,temp)

In [24]:
my_rdd = (
    lines_rdd
    .map(split_line)
    .filter(lambda x : "TMIN" in x[1])
    .map(lambda x : (x[0], x[2]))
    .reduceByKey(lambda x,y : min(x,y))
    .collect()
    #.min(key=lambda x: x[1])
    )

for result in my_rdd:
    print(f"{result[0]} Station reported {round(result[1],2)}C")

ITE00100554 Station reported 5.36C
EZE00100082 Station reported 7.7C


In [25]:
my_max_rdd = (
    lines_rdd
    .map(split_line)
    .filter(lambda x : "TMAX" in x[1])
    .map(lambda x : (x[0], x[2]))
    .reduceByKey(lambda x,y : max(x,y))
    .collect()
    #.min(key=lambda x: x[1])
    )

for result in my_max_rdd:
    print(f"{result[0]} Station reported {round(result[1],2)}C")

ITE00100554 Station reported 90.14C
EZE00100082 Station reported 90.14C


In [26]:
def another_split_line(line):
    return line.split(",")

In [33]:
my_ano_rdd = (
    sc.textFile("16_1800.csv")
    .map(another_split_line)
    )
my_ano_rdd.first()

['ITE00100554', '18000101', 'TMAX', '-75', '', '', 'E', '']

In [32]:
my_ano_rdd_2 = (
    sc.textFile("16_1800.csv")
    .flatMap(another_split_line)
    )
my_ano_rdd_2.first()

'ITE00100554'

In [38]:
my_data = sc.wholeT("19_Book.txt",encoding="utf-8")
my_data = (my_data
           .flatMap( lambda x: x.split(" "))
           .map(lambda x: (x,1))
           .reduceByKey(lambda x,y: x+y)
           )
my_data.top(5)
for word_tuple in my_data.collect():
    print(f"{word_tuple[0]} {word_tuple[1]}")

TypeError: SparkContext.textFile() got an unexpected keyword argument 'encoding'

: 